In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import spacy
nlp = spacy.load('en')

import re

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
#para = "What triggers an outbreak of the influenza virus? A new study of the flu in the 48 contiguous U.S. states, using data from the Atmospheric Infrared Sounder (AIRS) on NASA’s Aqua satellite, has found that the answer is closely tied to local weather – specifically, to low humidity – and varies from state to state. Average humidity varies widely across the United States, but even in the most humid states, it begins to drop as winter approaches. Researchers at NASA’s Jet Propulsion Laboratory in Southern California and the University of Southern California correlated AIRS measurements of water vapor in the lower atmosphere with flu case estimates for each week from 2003 to 2015. The researchers found that in each state, there is a specific level of low humidity that may signal a flu outbreak is imminent. When this threshold is crossed each year, a large increase in flu cases follows within two or three weeks, on average."
para = "On March 22, the newest U.S.-European sea level satellite, named Sentinel-6 Michael Freilich, became the official reference satellite for global sea level measurements. This means that sea surface height data collected by other satellites will be compared to the information produced by Sentinel-6 Michael Freilich to ensure their accuracy. Launched from Vandenberg Air Force Base in November 2020, the satellite is continuing a nearly 30-year legacy started by the TOPEX/Poseidon satellite, which began its mission to measure sea surface height in the early 1990s. A series of successor satellites have carried on the effort since then, with Sentinel-6 Michael Freilich being the most recent. Its twin, Sentinel-6B, is slated to launch in 2025. 'These missions, of which Sentinel-6 Michael Freilich is the latest, are the gold standard when it comes to sea level measurements, which are critical for understanding and monitoring climate change,' said Josh Willis; Sentinel-6 Michael Freilich project scientist at NASA’s Jet Propulsion Laboratory in Southern California."
para

"On March 22, the newest U.S.-European sea level satellite, named Sentinel-6 Michael Freilich, became the official reference satellite for global sea level measurements. This means that sea surface height data collected by other satellites will be compared to the information produced by Sentinel-6 Michael Freilich to ensure their accuracy. Launched from Vandenberg Air Force Base in November 2020, the satellite is continuing a nearly 30-year legacy started by the TOPEX/Poseidon satellite, which began its mission to measure sea surface height in the early 1990s. A series of successor satellites have carried on the effort since then, with Sentinel-6 Michael Freilich being the most recent. Its twin, Sentinel-6B, is slated to launch in 2025. 'These missions, of which Sentinel-6 Michael Freilich is the latest, are the gold standard when it comes to sea level measurements, which are critical for understanding and monitoring climate change,' said Josh Willis; Sentinel-6 Michael Freilich projec

In [ ]:
def ie_preprocess(document):
  sentences = nltk.sent_tokenize(document)
  sentences = [nltk.word_tokenize(sent) for sent in sentences]
  sentences = [nltk.pos_tag(sent) for sent in sentences]
  return sentences

**Algorithm:**
1. came up with some predefined rules from manual analysis
2. used NER to ignore CARDINAL and DATE as they are most likely not keywords (however, a feature to look at what happend regarding climate on that date can be developed from it)
3. words with LOC also dont qualify as keyword according to the analysis
4. only identify unique words as keywords

**Extra fetaures:**
1. New features can be built with just some NER

In [ ]:
def get_chunks(para):

  grammar = '\n'.join([
    'My_chunk: {<TO>+<JJ>+<NNS|NN|IN>+}',
    'My_chunk: {<NNP>+<NNS>+}',
    'My_chunk: {<NNP>+<VBD>+<NNP>+<NN>+}',
    'My_chunk: {<NN|NNP>+<NN|NNP>+}', 
    'My_chunk: {<DT>+<VBG>+<NNS>+}',
    'My_chunk: {<NNS>+<JJ>+<NN>+}',
    'My_chunk: {<VB>+<NN|NNS>+}',
    'My_chunk: {<NN>+<NNP>+<NN|NNS>+}',
    'My_chunk: {<,><NNS|NN|NNP>+<,>}',
    ])

  sentences = sent_tokenize(para)
  all_keywords = []
  for s in sentences:
    s = re.sub(r'[^\w\s]', '', s)
    print(s)
    ner_dict = {}
    doc = nlp(s)
    for ent in doc.ents:
      if (ent.label_ != ("CARDINAL" or "DATE")):
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        ner_dict[ent.text] = ent.label_
    print(ner_dict)

    sent = ie_preprocess(s)[0]
    print(sent)
    cp = nltk.RegexpParser(grammar)
    result = cp.parse(sent)

    for a in result:
      if isinstance(a, nltk.tree.Tree):
          if a.label() == "My_chunk":
              word = " ".join([lf[0] for lf in a.leaves()])
              if word not in all_keywords:
                if word in ner_dict:
                  if ner_dict[word] != "LOC":
                    all_keywords.append(word)
                    print(word)
                else:
                  all_keywords.append(word)
                  print(word)
    print()

  #print(result)

In [ ]:
get_chunks(para)

On March 22 the newest USEuropean sea level satellite named Sentinel6 Michael Freilich became the official reference satellite for global sea level measurements
{'March 22': 'DATE', 'USEuropean': 'GPE', 'Sentinel6': 'NORP', 'Michael Freilich': 'PERSON'}
[('On', 'IN'), ('March', 'NNP'), ('22', 'CD'), ('the', 'DT'), ('newest', 'JJS'), ('USEuropean', 'JJ'), ('sea', 'NN'), ('level', 'NN'), ('satellite', 'NN'), ('named', 'VBN'), ('Sentinel6', 'NNP'), ('Michael', 'NNP'), ('Freilich', 'NNP'), ('became', 'VBD'), ('the', 'DT'), ('official', 'JJ'), ('reference', 'NN'), ('satellite', 'NN'), ('for', 'IN'), ('global', 'JJ'), ('sea', 'NN'), ('level', 'NN'), ('measurements', 'NNS')]
sea level satellite
Sentinel6 Michael Freilich
reference satellite
sea level

This means that sea surface height data collected by other satellites will be compared to the information produced by Sentinel6 Michael Freilich to ensure their accuracy
{'Sentinel6': 'NORP', 'Michael Freilich': 'PERSON'}
[('This', 'DT'), ('mean